In [ ]:
# reference : https://github.com/IyatomiLab/LeafGAN

In [1]:
!nvidia-smi

Sat May 28 13:56:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import shutil
from glob import glob
import os
from natsort import natsorted
import pandas as pd

In [3]:
!unzip -qq /content/drive/MyDrive/4-1_Capstone/CapstoneDataset/images_concatenate.zip -d /content/capstone_images
capstone_images = natsorted(glob(os.path.join("/content/capstone_images", '*')))
capstone_label_df = pd.read_csv("/content/drive/MyDrive/4-1_Capstone/CapstoneDataset/Capstone_labels.csv")

SAMPLES = 1000
os.system("unzip -qq /content/drive/MyDrive/4-1_Capstone/AI_HubDataset/images" + str(SAMPLES) + ".zip -d /content/aihub_images")
aihub_images = natsorted(glob(os.path.join("/content/aihub_images/images" + str(SAMPLES), '*')))
aihub_label_df = pd.read_csv("/content/aihub_images/AI_Label_" + str(SAMPLES)  +".csv")

images = capstone_images + aihub_images
label_df = pd.concat([capstone_label_df, aihub_label_df]).fillna(0.).reset_index(drop = True)

In [4]:
# !rm -rf /content/dataset

In [6]:
!git clone https://github.com/IyatomiLab/LeafGAN
%cd /content/LeafGAN
!pip install -r requirements.txt -qq
!pip install dominate -qq

Cloning into 'LeafGAN'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 239 (delta 51), reused 51 (delta 51), pack-reused 176
Receiving objects: 100% (239/239), 5.21 MiB | 26.93 MiB/s, done.
Resolving deltas: 100% (127/127), done.
/content/LeafGAN
     |████████████████████████████████| 676 kB 7.8 MB/s 
     |████████████████████████████████| 47.8 MB 91.5 MB/s 
     |████████████████████████████████| 38.1 MB 5.8 MB/s 
ERROR: Could not find a version that satisfies the requirement skimage>=0.16.1 (from versions: 0.0)
ERROR: No matching distribution found for skimage>=0.16.1


In [5]:
import sys
sys.path.append("/content/drive/MyDrive/4-1_Capstone/")
from utils import get_disease_healthy_map 
disease_healthy_map = get_disease_healthy_map()

In [8]:
import shutil
import numpy as np

def visualize(disease_label, threshold = 0.35):
    global images, SAMPLES
    healthy_label = disease_healthy_map[disease_label]

    healthy_images = list(filter(lambda f : f.split('/')[-1].split('_')[0] == healthy_label, images))
    disease_images = list(filter(lambda f : f.split('/')[-1].split('_')[0] == disease_label, images))
    
    if os.path.exists("/content/dataset"):
        os.system("rm -rf /content/dataset")
        
    os.mkdir("/content/dataset")
    os.mkdir("/content/dataset/trainA")
    os.mkdir("/content/dataset/testA")
    os.mkdir("/content/dataset/trainB")
    os.mkdir("/content/dataset/testB")

    healthy_len = len(healthy_images)
    disease_len = len(disease_images)

    healthy_images = np.array(healthy_images)
    disease_images = np.array(disease_images)

    from sklearn.model_selection import train_test_split
    train_A_i, test_A_i, _, _ = train_test_split(list(range(healthy_len)), list(range(healthy_len)), random_state=0)
    train_B_i, test_B_i, _, _ = train_test_split(list(range(disease_len)), list(range(disease_len)), random_state=0)

    import shutil
    for image in healthy_images[train_A_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/trainA", file_name)

        shutil.move(image, dst)

    for image in healthy_images[test_A_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/testA", file_name)

        shutil.move(image, dst)

    for image in disease_images[train_B_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/trainB", file_name)

        shutil.move(image, dst)

    for image in disease_images[test_B_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/testB", file_name)

        shutil.move(image, dst)

    # os.system(f"python prepare_mask.py --source /content/dataset --pretrain_path /content/drive/MyDrive/4-1_Capstone/LFLSeg_resnet101.pth --image_size 224 --threshold {threshold}")

    # from glob import glob
    # from PIL import Image
    # import matplotlib.pyplot as plt

    # imgs1      = glob("/content/dataset/testA/*")
    # imgs1_mask = glob("/content/dataset/testA_mask/*")

    # cnt = 0
    # for img1, img1_mask in zip(imgs1, imgs1_mask):    
    #     plt.subplot(1, 2, 1)
    #     img1 = Image.open(img1)
    #     plt.imshow(img1)
        
    #     plt.subplot(1, 2, 2)
    #     img1_mask = Image.open(img1_mask)
    #     plt.imshow(img1_mask)
    #     plt.show()
        # if np.array(img1_mask).sum() == 0: cnt+=1


In [11]:
visualize('47', threshold = '.35')

In [12]:
import argparse
from pathlib import Path

import cv2
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms
from tqdm import tqdm

from models.grad_cam import GradCAM


def parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser()
    parser.add_argument("--source", type=str, default=None, help="leaf GAN's dataset")
    parser.add_argument(
        "--threshold",
        "-t",
        type=float,
        default=0.35,
        help="heatmap threshold",
    )
    parser.add_argument(
        "--pretrain_path",
        "-p",
        type=str,
        default=None,
        help="pretrain model path of LFLSeg ",
    )
    parser.add_argument("--image_size", "-i", type=int, help="size of image")
    args = parser.parse_args()
    return args


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, paths, image_size):
        self.paths = paths
        self.transform = transforms.Compose(
            [
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ]
        )

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx])
        return self.transform(img)


def get_heatmap(model, loader, device):
    heatmaps = []
    for img in tqdm(loader):
        img = img.to(device)
        with torch.enable_grad():
            _ = model.forward(img)
            model.backward(idx=0)
        heatmap = model.generate(target_layer="layer4.2")
        heatmaps.append(heatmap)
    return heatmaps

def visualize_heatmap(heatmaps, image_size, paths):
    import matplotlib.pyplot as plt
    for heatmap, p in zip(heatmaps, paths):
        mask = cv2.resize(heatmap, dsize=(image_size, image_size))
        p = Image.open(p)
        plt.figure(figsize=(15, 3))

        thresholds = [0.01, 0.05, 0.1, 0.35]
        plt.subplot(1, len(thresholds) + 1, 1)
        plt.imshow(p)
        for i, threshold in enumerate(thresholds):
            mask_ = (mask >= threshold).astype(int)
            mask_ = np.repeat(mask_[:, :, np.newaxis], 3, axis=2) * 255
            mask_img = Image.fromarray(mask_.astype(np.uint8))
            plt.subplot(1, len(thresholds) + 1, i+2)
            plt.imshow(mask_img)
            plt.title(threshold)

        plt.show()
            # mask_img.save(out_dir / p.name)

device = "cuda" if torch.cuda.is_available() else "cpu"

# setting LFLSeg Model
segResNet = models.resnet101()
num_ftrs = segResNet.fc.in_features
segResNet.fc = nn.Linear(num_ftrs, 3)
pretrain_path = "/content/drive/MyDrive/4-1_Capstone/LFLSeg_resnet101.pth"
segResNet.load_state_dict(torch.load(pretrain_path), strict=True)
segResNet.to(device)
segResNet.eval()

netLFLSeg = GradCAM(model=segResNet)

# setup mask data folder
source = "/content/dataset"
data_root = Path(source)
dataset_dirs = [p for p in data_root.glob("*") if "mask" not in str(p)]
mask_dataset_dirs = []

for data_dir in dataset_dirs:
    out_dir = data_root / f"{data_dir.name}_mask"
    out_dir.mkdir(exist_ok=True)
    mask_dataset_dirs.append(out_dir)

# get_mask
image_size = 224
for source_dir, out_dir in zip(dataset_dirs, mask_dataset_dirs):
    print(f"##### {source_dir.name} #####")
    paths = list(source_dir.glob("*"))
    loader = torch.utils.data.DataLoader(
        MyDataset(paths, image_size),
        shuffle=False,
        batch_size=1,
        num_workers=1,
    )
    heatmaps = get_heatmap(netLFLSeg, loader, device)
    visualize_heatmap(heatmaps, 224, paths = paths)

Output hidden; open in https://colab.research.google.com to view.